In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ----------------------------------- NOTEBOOK SETUP --------------------------------------------------------- #
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from math import sqrt
import time
import os
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import minmax_scale

# Setting plot style
plt.style.use('ggplot')

# naver 데이터 scale 처리
매장마다 리뷰 수가 다르기에 회귀는 이에 영향을 많이 받을 수 있어 total review로 나누어 scale을 진행하였음.

In [ ]:
naver_more = pd.read_csv('/content/drive/MyDrive/modeling/github/data/naver_data/naver_review_name.csv')
naver_more.drop(['search_name','address','road_address','near_station','station_distance'],axis=1,inplace=True)
naver_more.columns

Index(['total_review', '"커피가 맛있어요"', '"친절해요"', '"음료가 맛있어요"', '"디저트가 맛있어요"',
       '"매장이 청결해요"', '"대화하기 좋아요"', '"집중하기 좋아요"', '"인테리어가 멋져요"', '"좌석이 편해요"',
       '"화장실이 깨끗해요"', '"특별한 메뉴가 있어요"', '"가성비가 좋아요"', '"뷰가 좋아요"', '"주차하기 편해요"',
       '"사진이 잘 나와요"', 'cafe'],
      dtype='object')

In [ ]:
columns = ['"커피가 맛있어요"', '"친절해요"', '"음료가 맛있어요"', '"디저트가 맛있어요"',
       '"매장이 청결해요"', '"대화하기 좋아요"', '"집중하기 좋아요"', '"인테리어가 멋져요"', '"좌석이 편해요"',
       '"화장실이 깨끗해요"', '"특별한 메뉴가 있어요"', '"가성비가 좋아요"', '"뷰가 좋아요"', '"주차하기 편해요"',
       '"사진이 잘 나와요"']
for column in columns:
  naver_more[column] = naver_more[column]/naver_more['total_review']
  print(column, '열에 대해 전체로 나누기 완료')

"커피가 맛있어요" 열에 대해 전체로 나누기 완료
"친절해요" 열에 대해 전체로 나누기 완료
"음료가 맛있어요" 열에 대해 전체로 나누기 완료
"디저트가 맛있어요" 열에 대해 전체로 나누기 완료
"매장이 청결해요" 열에 대해 전체로 나누기 완료
"대화하기 좋아요" 열에 대해 전체로 나누기 완료
"집중하기 좋아요" 열에 대해 전체로 나누기 완료
"인테리어가 멋져요" 열에 대해 전체로 나누기 완료
"좌석이 편해요" 열에 대해 전체로 나누기 완료
"화장실이 깨끗해요" 열에 대해 전체로 나누기 완료
"특별한 메뉴가 있어요" 열에 대해 전체로 나누기 완료
"가성비가 좋아요" 열에 대해 전체로 나누기 완료
"뷰가 좋아요" 열에 대해 전체로 나누기 완료
"주차하기 편해요" 열에 대해 전체로 나누기 완료
"사진이 잘 나와요" 열에 대해 전체로 나누기 완료


In [ ]:
naver_more.head()

,total_review,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",cafe
0,18,0.388889,0.277778,0.277778,0.333333,0.277778,0.000000,0.000000,0.000000,0.055556,0.111111,0.111111,0.000000,0.000000,0.055556,0.000000,버블베어 방화동
1,284,0.024648,0.197183,0.260563,0.130282,0.140845,0.073944,0.169014,0.028169,0.116197,0.007042,0.007042,0.028169,0.049296,0.000000,0.003521,커피빈코리아낙성대역점 코리아낙성대역점 봉천동
2,134,0.298507,0.104478,0.455224,0.291045,0.014925,0.000000,0.000000,0.007463,0.000000,0.000000,0.156716,0.246269,0.000000,0.007463,0.000000,노란고무줄 대치동
3,100,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,0.100000,0.040000,0.050000,0.000000,0.040000,커피랩 서교동
4,140,0.457143,0.100000,0.221429,0.092857,0.107143,0.207143,0.050000,0.014286,0.107143,0.035714,0.035714,0.042857,0.021429,0.000000,0.007143,커핀그루나루신천점 신천점 잠실동


In [ ]:
ratings_df = pd.read_csv('/data/kakao_data/ratings.csv')
ratings_df = ratings_df[['user_id', 'cafe', 'rating']]

In [ ]:
len(ratings_df)

49088

In [ ]:
ratings_df = pd.merge(ratings_df, naver_more['cafe'],how='inner', on='cafe')
naver_more = naver_more.drop_duplicates()
print(len(naver_more))
naver_more = pd.merge(naver_more, ratings_df.groupby('cafe',as_index=False).count()['cafe'], how='inner',on='cafe')
print('rating의 총 행 수',len(ratings_df))
print(len(naver_more))

7660
rating의 총 행 수 35032
4508


In [ ]:
find = ratings_df.groupby('user_id', as_index=False).count()
twoormore = find[find['rating']>1]['user_id']
print('2개 이상의 별점을 준 user 수',len(twoormore))
twoormore.head()
ratings_df = pd.merge(ratings_df,twoormore,how='inner',on=['user_id'])
print('rating의 총 행 수',len(ratings_df))

2개 이상의 별점을 준 user 수 4708
rating의 총 행 수 15289


In [ ]:
naver_more = pd.merge(naver_more, ratings_df.groupby('cafe',as_index=False).count()['cafe'], how='inner',on='cafe')
print('2개 이상의 별점을 준 user에 대응하는 카페 수',len(naver_more))
# 헉 엄청줄었어....

2개 이상의 별점을 준 user에 대응하는 카페 수 3444


# model train / test

In [ ]:
# Splitting the ratings dataset into the feature set (X) and target labels (y)
X = ratings_df.drop(columns='rating')
y = ratings_df["rating"].values  # The movie ratings are the target variables we want to predict

# Preparing train, validation and test datasets.
# I have chosen a split ratio of 70%, 15%, 15%, because I want a somewhat large training set at the cost of a
# smaller validation and test set. I do not think that a smaller validation (or test) dataset will negatively
# impact the generalization ability of the chosen models, because I am only using rather simple ML models
# with few hyperparamaters.
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=101)

# Creating a complete training dataset with X_train and y_train
train_df = X_train.copy()
train_df["rating"] = y_train

train_df

,user_id,cafe,rating
11685,15ghb6f,와플대학노원역캠퍼스 노원역캠퍼스 상계동,1.0
6610,1kkavti,컴포즈커피 사당임하빌딩점 사당동,1.0
14392,uhf7ab,빨라쪼델프레도 코엑스몰점 삼성동,4.0
1726,oaj2pu,젤라떼리아도도 송파동,5.0
10808,bep1st,투썸플레이스 서울역센트럴자이점 만리동2가,4.0
...,...,...,...
5695,mv7s2v,212베이크샵 마곡동,5.0
8006,108mfu0,센터커피 서울숲점 성수동1가,3.0
1361,e4mb02,바렌나에스프레소바 송파동,5.0
1547,1gtnudn,테라로사 포스코센터점 대치동,5.0


In [ ]:
content_train_df = pd.merge(naver_more,train_df, how='inner', on='cafe')
content_train_df.drop('cafe',axis=1,inplace=True)
content_train_df.drop('total_review',axis=1, inplace=True)
content_train_df

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""",...,"""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",사주카페,생과일주스전문점,애견카페,전통찻집/인삼찻집,커피전문점/카페/다방,user_id,rating
0,0.388889,0.277778,0.277778,0.333333,0.277778,0.000000,0.000000,0.000000,0.055556,0.111111,...,0.00,0.055556,0.00,False,False,False,False,True,7t5a08,4.0
1,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.05,0.000000,0.04,False,False,False,False,True,15s3i66,3.0
2,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.05,0.000000,0.04,False,False,False,False,True,kf1se6,5.0
3,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.05,0.000000,0.04,False,False,False,False,True,mj5evf,4.0
4,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.05,0.000000,0.04,False,False,False,False,True,s4t1h2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10697,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.00,0.000000,0.00,False,False,False,False,True,11b3qbh,4.0
10698,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.00,0.000000,0.00,False,False,False,False,True,dnqkqi,1.0
10699,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.00,0.000000,0.00,False,False,False,False,True,13unamf,5.0
10700,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.00,0.000000,0.00,False,False,False,False,True,vgmvkf,3.0


In [ ]:
# Creating a list of lists with the target attribute (movie ratings), grouped by userID
y_grouped_by_user = content_train_df.groupby(["user_id"])
y_train_listed = []

for i, j in y_grouped_by_user:
    y_train_listed.append(j["rating"].values)  # Extract target feature (ratings) from sorted data
    
# Target attributes for the first user
y_train_listed[0]

array([5., 1.])

In [ ]:
# Creating a list of dataframes with the feature set (movie info about genres and release year), grouped by userID
content_train_df.drop(columns='rating', inplace=True) #이건 y값인데 y값을 위에서 정의를 해줬으니 없애주어야
x_grouped_by_user = content_train_df.groupby(["user_id"])
x_train_listed = []

for user_id, group in x_grouped_by_user:
    x_train_listed.append(group.drop(columns='user_id'))
    
# The feature set for the first user
x_train_listed[0]

# 앞의 인덱스는 영화 이름 아니고 그냥 인덱스임!!

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",사주카페,생과일주스전문점,애견카페,전통찻집/인삼찻집,커피전문점/카페/다방
9505,0.0,0.003036,0.001012,0.008097,0.007085,0.075911,0.035425,0.075911,0.052632,0.053644,0.050607,0.018219,0.014170,0.029352,0.011134,False,False,False,False,True
9915,0.0,0.186544,0.000000,0.079511,0.110092,0.009174,0.006116,0.018349,0.033639,0.000000,0.116208,0.073394,0.006116,0.000000,0.003058,False,False,False,False,True


In [ ]:
# Creating a new dataframe for the movies data
all_cafes = naver_more.drop(columns=['cafe','total_review'])
all_cafes

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",사주카페,생과일주스전문점,애견카페,전통찻집/인삼찻집,커피전문점/카페/다방
0,0.388889,0.277778,0.277778,0.333333,0.277778,0.000000,0.000000,0.000000,0.055556,0.111111,0.111111,0.000000,0.000000,0.055556,0.000000,False,False,False,False,True
1,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,0.100000,0.040000,0.050000,0.000000,0.040000,False,False,False,False,True
2,0.457143,0.100000,0.221429,0.092857,0.107143,0.207143,0.050000,0.014286,0.107143,0.035714,0.035714,0.042857,0.021429,0.000000,0.007143,False,False,False,False,True
3,0.011628,0.186047,0.156977,0.081395,0.127907,0.063953,0.034884,0.046512,0.063953,0.011628,0.023256,0.162791,0.017442,0.005814,0.005814,False,False,False,False,True
4,0.019048,0.185714,0.166667,0.142857,0.109524,0.142857,0.071429,0.033333,0.076190,0.000000,0.033333,0.014286,0.080952,0.009524,0.019048,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,0.014493,0.149758,0.227053,0.038647,0.106280,0.096618,0.048309,0.009662,0.048309,0.004831,0.082126,0.217391,0.009662,0.004831,0.004831,False,False,False,False,True
3440,0.000000,0.182648,0.173516,0.305936,0.164384,0.182648,0.036530,0.073059,0.127854,0.036530,0.031963,0.105023,0.031963,0.013699,0.004566,False,False,False,False,True
3441,0.005291,0.243386,0.201058,0.089947,0.132275,0.158730,0.111111,0.026455,0.111111,0.015873,0.026455,0.148148,0.010582,0.010582,0.031746,False,False,False,False,True
3442,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,0.074074,0.037037,0.000000,0.000000,0.000000,False,False,False,False,True


In [ ]:
# ----------- PREPARING VALIDATION DATA ----------- #
# Creating a 2 dimensional matrix for the validation data in order to make it easier to calculate RMSE.

# Listing the user ID's in the same order as in the grouped dataframes
user_ids = []
for user_id, group in x_grouped_by_user:
    user_ids.append(user_id)
    
# Listing the movie IDs in the same order as in the movies dataset
cafe_ids = naver_more["cafe"].values

# Creating the matrix. Axis 1: User IDs, Axis 2: Movie IDs. Elements: True ratings from validation data
df_val = X_val_and_test.copy()
df_val["rating"] = y_val_and_test
validation_matrix = pd.DataFrame(index=user_ids, columns=cafe_ids)  # Starting with an empty matrix
for array in df_val.to_records():  # Filling in the true ratings as elements
    user = array['user_id']
    cafe = array['cafe']
    true_rating = array['rating']
    if user not in user_ids:
      continue
    validation_matrix.loc[user][cafe] = true_rating
    
validation_matrix

,버블베어 방화동,커피랩 서교동,커핀그루나루신천점 신천점 잠실동,이디야커피 신길역점 신길동,커피빈 코리아청담성당점 청담동,스타벅스 동숭로아트점 동숭동,가비애 서교동,훈고링고브레드 성산동,스타벅스남부터미널2점 남부터미널2점 서초동,스타벅스 현대목동점 목동,...,이디야커피 선릉사거리점 역삼동,이디야커피 당곡사거리점 봉천동,스타벅스 삼성현대힐점 삼성동,이디야커피 문정테라타워점 문정동,스타벅스 안국점 재동,이디야커피 동자동점 동자동,커피나무 보라매역점 신길동,이디야커피 논골사거리점 금호동1가,퍼블릭맨션 목동,이디야커피 굽은다리역점 천호동
100ck0j,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100i22n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100ooji,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100q04c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100rluq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vtt4u1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vtth18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vua8j4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vur3vt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
x_train_listed[0]

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",사주카페,생과일주스전문점,애견카페,전통찻집/인삼찻집,커피전문점/카페/다방
9505,0.0,0.003036,0.001012,0.008097,0.007085,0.075911,0.035425,0.075911,0.052632,0.053644,0.050607,0.018219,0.014170,0.029352,0.011134,False,False,False,False,True
9915,0.0,0.186544,0.000000,0.079511,0.110092,0.009174,0.006116,0.018349,0.033639,0.000000,0.116208,0.073394,0.006116,0.000000,0.003058,False,False,False,False,True


## Support Vector Regression fine-tuning

In [ ]:
import warnings
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings('ignore')
CBF_models_listed = []
RMSE_CBF_listed = []
MAE_CBF_listed = []
for a in (0.1, 1, 10, 100, 1000):
  for b in (0.1, 1, 10, 100, 1000):
    ml_alg = SVR(C=a, gamma=b)
    # Create an empty list for predictions
    CBF_predictions = []

    # For each user in the training dataset:
    for i, x in enumerate(x_train_listed):
        # Fit a machine learning model
        ml_alg.fit(x_train_listed[i], y_train_listed[i])
        # Predict all the ratings for this user for all movies
        prediction = ml_alg.predict(all_cafes)
        prediction = np.clip(prediction, 1, 5)  # Predictions must be minimum 1, maximum 5
        # Append all the predictions to the predictions list
        CBF_predictions.append(prediction)

        if i%1000==0:
          print('train', i, 'finished!')

    # Create a dataframe with the predictions
    df_predict = pd.DataFrame(CBF_predictions, index=user_ids, columns=cafe_ids)

    # Create a dataframe with only the predictions for the movies-user combinations that appear in the validation set
    num_actual = validation_matrix.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]
    num_predict = df_predict.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]

    # Calculate the RMSE for the content-based filtering model and add the result to the lists
    RMSE_CBF_listed.append(sqrt(mean_squared_error(num_predict, num_actual)))
    MAE_CBF_listed.append(mean_absolute_error(num_predict, num_actual))
    CBF_models_listed.append(ml_alg)

    print(ml_alg,'finished!')

RMSE_CBF_df = pd.DataFrame({"Model": CBF_models_listed, "RMSE": RMSE_CBF_listed, "MAE":MAE_CBF_listed})
print("RMSE of different content-based filtering models with all reviews")
RMSE_CBF_df

train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
SVR(C=0.1, gamma=0.1) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
SVR(C=0.1, gamma=1) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
SVR(C=0.1, gamma=10) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
SVR(C=0.1, gamma=100) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
SVR(C=0.1, gamma=1000) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
SVR(C=1, gamma=0.1) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
SVR(C=1, gamma=1) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000

,Model,RMSE,MAE
0,"SVR(C=0.1, gamma=0.1)",1.515358,0.979195
1,"SVR(C=0.1, gamma=1)",1.513834,0.979436
2,"SVR(C=0.1, gamma=10)",1.512222,0.980728
3,"SVR(C=0.1, gamma=100)",1.512116,0.981158
4,"SVR(C=0.1, gamma=1000)",1.511966,0.981092
5,"SVR(C=1, gamma=0.1)",1.513098,0.979207
6,"SVR(C=1, gamma=1)",1.507763,0.981651
7,"SVR(C=1, gamma=10)",1.500379,0.988967
8,"SVR(C=1, gamma=100)",1.494487,0.989807
9,"SVR(C=1, gamma=1000)",1.493348,0.990191


## Random Forest Regressor fine-tuning

In [ ]:
import warnings
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings('ignore')
CBF_models_listed = []
RMSE_CBF_listed = []
MAE_CBF_listed = []
for a in (10,50,100):
    ml_alg = RandomForestRegressor(n_estimators=a, n_jobs=3, max_features="auto", random_state=0)
    # Create an empty list for predictions
    CBF_predictions = []

    # For each user in the training dataset:
    for i, x in enumerate(x_train_listed):
        # Fit a machine learning model
        ml_alg.fit(x_train_listed[i], y_train_listed[i])
        # Predict all the ratings for this user for all movies
        prediction = ml_alg.predict(all_cafes)
        prediction = np.clip(prediction, 1, 5)  # Predictions must be minimum 1, maximum 5
        # Append all the predictions to the predictions list
        CBF_predictions.append(prediction)

        if i%1000==0:
          print('train', i, 'finished!')

    # Create a dataframe with the predictions
    df_predict = pd.DataFrame(CBF_predictions, index=user_ids, columns=cafe_ids)

    # Create a dataframe with only the predictions for the movies-user combinations that appear in the validation set
    num_actual = validation_matrix.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]
    num_predict = df_predict.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]

    # Calculate the RMSE for the content-based filtering model and add the result to the lists
    RMSE_CBF_listed.append(sqrt(mean_squared_error(num_predict, num_actual)))
    MAE_CBF_listed.append(mean_absolute_error(num_predict, num_actual))
    CBF_models_listed.append(ml_alg)

    print(ml_alg,'finished!')

RMSE_CBF_df = pd.DataFrame({"Model": CBF_models_listed, "RMSE": RMSE_CBF_listed, "MAE":MAE_CBF_listed})
print("RMSE of different content-based filtering models with all reviews")
RMSE_CBF_df

train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
RandomForestRegressor(n_estimators=10, n_jobs=3, random_state=0) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
RandomForestRegressor(n_estimators=50, n_jobs=3, random_state=0) finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
RandomForestRegressor(n_jobs=3, random_state=0) finished!
RMSE of different content-based filtering models with all reviews


,Model,RMSE,MAE
0,"(DecisionTreeRegressor(max_features='auto', ra...",1.519498,1.017322
1,"(DecisionTreeRegressor(max_features='auto', ra...",1.504708,1.010308
2,"(DecisionTreeRegressor(max_features='auto', ra...",1.505878,1.011563


## Linear regression RMSE 확인

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# check the scale of the features or consider increasing regularisation
# ----------- CREATING THE DIFFERENT CONTENT-BASED FILTERING MODELS ----------- #
# Assigning the different machine learning algorithms to be implemented in the models (incl. hyperparameters) to a dictionary
ml_algorithms = {"Linear regression": LinearRegression(), 
                 "Lasso": Lasso(alpha=0.01, max_iter=10000), 
                 #"KNN_7": KNeighborsRegressor(n_neighbors=7),
                 #"RFR": RandomForestRegressor(n_estimators=50, n_jobs=3, max_features="auto", random_state=0),
                 #"SVR": SVR(C=10, gamma=1000)
                 }

# Saving lists that I later use to construct a dataframe containing the performances of the models
CBF_models_listed = []
RMSE_CBF_listed = []
MAE_CBF_listed = []

# For every machine learning algorithm in the dictionary:
for name, ml_alg in ml_algorithms.items():
    # Create an empty list for predictions
    CBF_predictions = []

    # For each user in the training dataset:
    for i, x in enumerate(x_train_listed):
        # Fit a machine learning model
        ml_alg.fit(x_train_listed[i], y_train_listed[i])
        # Predict all the ratings for this user for all movies
        prediction = ml_alg.predict(all_cafes)
        prediction = np.clip(prediction, 1, 5)  # Predictions must be minimum 1, maximum 5
        # Append all the predictions to the predictions list
        CBF_predictions.append(prediction)

        if i%1000==0:
          print('train', i, 'finished!')

    # Create a dataframe with the predictions
    df_predict = pd.DataFrame(CBF_predictions, index=user_ids, columns=cafe_ids)

    # Create a dataframe with only the predictions for the movies-user combinations that appear in the validation set
    num_actual = validation_matrix.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]
    num_predict = df_predict.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]

    # Calculate the RMSE for the content-based filtering model and add the result to the lists
    RMSE_CBF_listed.append(sqrt(mean_squared_error(num_predict, num_actual)))
    CBF_models_listed.append(name)
    MAE_CBF_listed.append(mean_absolute_error(num_predict, num_actual))

    print(ml_alg,'finished!')

train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
LinearRegression() finished!
train 0 finished!
train 1000 finished!
train 2000 finished!
train 3000 finished!
train 4000 finished!
Lasso(alpha=0.01, max_iter=10000) finished!


In [ ]:
# Printing the results
RMSE_CBF_df = pd.DataFrame({"Model": CBF_models_listed, "RMSE": RMSE_CBF_listed, "MAE":MAE_CBF_listed})
print("RMSE and MAE of different content-based filtering models with users who rated more than two")
RMSE_CBF_df

RMSE and MAE of different content-based filtering models with users who rated more than two


,Model,RMSE,MAE
0,Linear regression,1.663399,1.108812
1,Lasso,1.615885,1.069656


In [ ]:
df_predict

,버블베어 방화동,커피랩 서교동,커핀그루나루신천점 신천점 잠실동,이디야커피 신길역점 신길동,커피빈 코리아청담성당점 청담동,스타벅스 동숭로아트점 동숭동,가비애 서교동,훈고링고브레드 성산동,스타벅스남부터미널2점 남부터미널2점 서초동,스타벅스 현대목동점 목동,...,이디야커피 선릉사거리점 역삼동,이디야커피 당곡사거리점 봉천동,스타벅스 삼성현대힐점 삼성동,이디야커피 문정테라타워점 문정동,스타벅스 안국점 재동,이디야커피 동자동점 동자동,커피나무 보라매역점 신길동,이디야커피 논골사거리점 금호동1가,퍼블릭맨션 목동,이디야커피 굽은다리역점 천호동
100ck0j,3.000000,3.000000,3.000000,3.000000,3.000000,3.000018,3.000000,3.000000,3.000322,3.047216,...,3.000000,3.000000,3.133339,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
100i22n,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
100ooji,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
100q04c,3.522222,3.522222,3.522222,3.522222,3.522222,3.522255,3.522222,3.522222,3.524642,3.522276,...,3.522222,3.522222,3.522224,3.522222,3.524248,3.522222,3.522222,3.522222,3.522222,3.522222
100rluq,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vtt4u1,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
vtth18,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
vua8j4,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
vur3vt,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


# full data에 대한 fit_transform

가장 성능이 좋았던 두 모델에 대한 fit_transform을 진행합니다.  
데이터가 달라질 경우 성능이 좋은 모델 또한 달라질 수 있습니다.

**유의사항**  
- fit_transform 한 뒤의 결과물을 csv로 저장할 때 용량을 많이 차지합니다.
- 모든 예측값에 대한 저장이 불필요한 경우, 같은 폴더에 있는 'CBF_ML_recommending.ipynb' 파일의 내용을 복사 붙여넣기하여, 파일 저장하고 그를 새로 불러오는 과정 없이 진행해주시면 됩니다.  
  - 이 경우 변수명이 다르니 이에 유의해주세요

In [ ]:
X = ratings_df.drop(columns='rating')
y = ratings_df["rating"].values

In [ ]:
input_X = X.copy()
input_X["rating"] = y

In [ ]:
content_input_X = pd.merge(naver_more,input_X, how='inner', on='cafe')
content_input_X.drop('cafe',axis=1,inplace=True)
content_input_X.drop('total_review',axis=1, inplace=True)
content_input_X

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""",...,"""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",사주카페,생과일주스전문점,애견카페,전통찻집/인삼찻집,커피전문점/카페/다방,user_id,rating
0,0.388889,0.277778,0.277778,0.333333,0.277778,0.000000,0.000000,0.000000,0.055556,0.111111,...,0.000000,0.055556,0.000000,False,False,False,False,True,7t5a08,4.0
1,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.050000,0.000000,0.040000,False,False,False,False,True,bfpnu9,5.0
2,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.050000,0.000000,0.040000,False,False,False,False,True,g1ubpf,4.0
3,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.050000,0.000000,0.040000,False,False,False,False,True,kf1se6,5.0
4,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,...,0.050000,0.000000,0.040000,False,False,False,False,True,192p5h3,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15284,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.000000,0.000000,0.000000,False,False,False,False,True,15ss2hm,5.0
15285,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.000000,0.000000,0.000000,False,False,False,False,True,17e9tba,5.0
15286,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.000000,0.000000,0.000000,False,False,False,False,True,dnqkqi,1.0
15287,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,...,0.000000,0.000000,0.000000,False,False,False,False,True,vv73qf,1.0


In [ ]:
# Creating a list of lists with the target attribute (movie ratings), grouped by userID
y_grouped_by_user = content_input_X.groupby(["user_id"])
y_listed = []

for i, j in y_grouped_by_user:
    y_listed.append(j["rating"].values)  # Extract target feature (ratings) from sorted data
    
# Target attributes for the first user
y_listed[0]

array([1., 5., 1.])

In [ ]:
# Creating a list of dataframes with the feature set (movie info about genres and release year), grouped by userID
content_input_X.drop(columns='rating', inplace=True) #이건 y값인데 y값을 위에서 정의를 해줬으니 없애주어야
x_grouped_by_user = content_input_X.groupby(["user_id"])
x_listed = []

for user_id, group in x_grouped_by_user:
    x_listed.append(group.drop(columns='user_id'))
    
# The feature set for the first user
x_listed[0]

# 앞의 인덱스는 영화 이름 아니고 그냥 인덱스임!!

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",사주카페,생과일주스전문점,애견카페,전통찻집/인삼찻집,커피전문점/카페/다방
7183,0.006772,0.176072,0.194131,0.018059,0.160271,0.164786,0.099323,0.079007,0.169300,0.047404,0.033860,0.029345,0.018059,0.000000,0.015801,False,False,False,False,True
13580,0.000000,0.003036,0.001012,0.008097,0.007085,0.075911,0.035425,0.075911,0.052632,0.053644,0.050607,0.018219,0.014170,0.029352,0.011134,False,False,False,False,True
14140,0.000000,0.186544,0.000000,0.079511,0.110092,0.009174,0.006116,0.018349,0.033639,0.000000,0.116208,0.073394,0.006116,0.000000,0.003058,False,False,False,False,True


In [ ]:
# Creating a new dataframe for the movies data
all_cafes = naver_more.drop(columns=['cafe','total_review'])
all_cafes

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",사주카페,생과일주스전문점,애견카페,전통찻집/인삼찻집,커피전문점/카페/다방
0,0.388889,0.277778,0.277778,0.333333,0.277778,0.000000,0.000000,0.000000,0.055556,0.111111,0.111111,0.000000,0.000000,0.055556,0.000000,False,False,False,False,True
1,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,0.100000,0.040000,0.050000,0.000000,0.040000,False,False,False,False,True
2,0.457143,0.100000,0.221429,0.092857,0.107143,0.207143,0.050000,0.014286,0.107143,0.035714,0.035714,0.042857,0.021429,0.000000,0.007143,False,False,False,False,True
3,0.011628,0.186047,0.156977,0.081395,0.127907,0.063953,0.034884,0.046512,0.063953,0.011628,0.023256,0.162791,0.017442,0.005814,0.005814,False,False,False,False,True
4,0.019048,0.185714,0.166667,0.142857,0.109524,0.142857,0.071429,0.033333,0.076190,0.000000,0.033333,0.014286,0.080952,0.009524,0.019048,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,0.014493,0.149758,0.227053,0.038647,0.106280,0.096618,0.048309,0.009662,0.048309,0.004831,0.082126,0.217391,0.009662,0.004831,0.004831,False,False,False,False,True
3440,0.000000,0.182648,0.173516,0.305936,0.164384,0.182648,0.036530,0.073059,0.127854,0.036530,0.031963,0.105023,0.031963,0.013699,0.004566,False,False,False,False,True
3441,0.005291,0.243386,0.201058,0.089947,0.132275,0.158730,0.111111,0.026455,0.111111,0.015873,0.026455,0.148148,0.010582,0.010582,0.031746,False,False,False,False,True
3442,0.629630,0.481481,0.333333,0.148148,0.259259,0.111111,0.074074,0.296296,0.037037,0.037037,0.074074,0.037037,0.000000,0.000000,0.000000,False,False,False,False,True


In [ ]:
# Listing the user ID's in the same order as in the grouped dataframes
user_ids = []
for user_id, group in x_grouped_by_user:
    user_ids.append(user_id)
    
# Listing the movie IDs in the same order as in the movies dataset
cafe_ids = naver_more["cafe"].values

## Support Vector Regression

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# check the scale of the features or consider increasing regularisation
# ----------- CREATING THE DIFFERENT CONTENT-BASED FILTERING MODELS ----------- #
# Assigning the different machine learning algorithms to be implemented in the models (incl. hyperparameters) to a dictionary
ml_alg = SVR(C=10, gamma=1000)


In [ ]:
CBF_predictions = []
for i, x in enumerate(x_listed):
        # Fit a machine learning model
        ml_alg.fit(x_listed[i], y_listed[i])
        # Predict all the ratings for this user for all movies
        prediction = ml_alg.predict(all_cafes)
        prediction = np.clip(prediction, 1, 5)  # Predictions must be minimum 1, maximum 5
        # Append all the predictions to the predictions list
        CBF_predictions.append(prediction)
df_predict = pd.DataFrame(CBF_predictions, index=user_ids, columns=cafe_ids)

In [ ]:
df_predict

,버블베어 방화동,커피랩 서교동,커핀그루나루신천점 신천점 잠실동,이디야커피 신길역점 신길동,커피빈 코리아청담성당점 청담동,스타벅스 동숭로아트점 동숭동,가비애 서교동,훈고링고브레드 성산동,스타벅스남부터미널2점 남부터미널2점 서초동,스타벅스 현대목동점 목동,...,이디야커피 선릉사거리점 역삼동,이디야커피 당곡사거리점 봉천동,스타벅스 삼성현대힐점 삼성동,이디야커피 문정테라타워점 문정동,스타벅스 안국점 재동,이디야커피 동자동점 동자동,커피나무 보라매역점 신길동,이디야커피 논골사거리점 금호동1가,퍼블릭맨션 목동,이디야커피 굽은다리역점 천호동
100ck0j,2.366667,2.366667,2.366667,2.366667,2.366667,2.366691,2.366667,2.366667,2.367095,2.429606,...,2.366667,2.366667,2.544408,2.366667,2.366667,2.366667,2.366667,2.366667,2.366667,2.366667
100i22n,3.550000,3.550000,3.550000,3.550000,3.550000,3.550000,3.550000,3.550000,3.550002,3.550183,...,3.550000,3.550000,3.550003,3.550000,3.550026,3.550000,3.550000,3.550000,3.550000,3.550000
100ooji,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
100q04c,3.590065,3.590065,3.590065,3.590065,3.590065,3.590092,3.590065,3.590065,3.592980,3.590406,...,3.590065,3.590065,3.590066,3.590065,3.596362,3.590065,3.590065,3.590065,3.590065,3.590065
100rluq,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.999985,2.999546,...,3.000000,3.000000,2.999999,3.000000,2.999971,3.000000,3.000000,3.000000,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vua8j4,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
vunrjc,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,...,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
vur3vt,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
vv73qf,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df_predict.to_csv('저장경로/CBF_total_two_SVR.csv', index=False)

## Random Forest Regression

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# check the scale of the features or consider increasing regularisation
# ----------- CREATING THE DIFFERENT CONTENT-BASED FILTERING MODELS ----------- #
# Assigning the different machine learning algorithms to be implemented in the models (incl. hyperparameters) to a dictionary
ml_alg = RandomForestRegressor(n_estimators=50, n_jobs=3, max_features="auto", random_state=0)


In [ ]:
CBF_predictions = []
for i, x in enumerate(x_listed):
        # Fit a machine learning model
        ml_alg.fit(x_listed[i], y_listed[i])
        # Predict all the ratings for this user for all movies
        prediction = ml_alg.predict(all_cafes)
        prediction = np.clip(prediction, 1, 5)  # Predictions must be minimum 1, maximum 5
        # Append all the predictions to the predictions list
        CBF_predictions.append(prediction)
df_predict = pd.DataFrame(CBF_predictions, index=user_ids, columns=cafe_ids)

In [ ]:
df_predict

,버블베어 방화동,커피랩 서교동,커핀그루나루신천점 신천점 잠실동,이디야커피 신길역점 신길동,커피빈 코리아청담성당점 청담동,스타벅스 동숭로아트점 동숭동,가비애 서교동,훈고링고브레드 성산동,스타벅스남부터미널2점 남부터미널2점 서초동,스타벅스 현대목동점 목동,...,이디야커피 선릉사거리점 역삼동,이디야커피 당곡사거리점 봉천동,스타벅스 삼성현대힐점 삼성동,이디야커피 문정테라타워점 문정동,스타벅스 안국점 재동,이디야커피 동자동점 동자동,커피나무 보라매역점 신길동,이디야커피 논골사거리점 금호동1가,퍼블릭맨션 목동,이디야커피 굽은다리역점 천호동
100ck0j,2.52,1.96,2.04,2.04,2.28,3.24,2.60,2.28,3.32,3.72,...,2.20,2.20,3.48,2.28,3.00,2.20,2.20,1.96,2.20,2.04
100i22n,3.44,3.08,3.24,3.60,3.28,3.72,3.60,3.24,3.88,3.76,...,3.56,3.32,3.68,3.60,3.92,3.28,3.20,3.40,3.12,3.36
100ooji,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
100q04c,3.10,3.24,3.64,3.66,3.86,3.86,3.78,3.16,3.88,3.98,...,3.70,3.68,3.88,3.68,3.90,3.60,3.60,3.50,3.40,3.66
100rluq,3.16,3.00,3.32,2.84,2.84,2.44,2.76,3.64,2.12,2.12,...,2.84,3.24,2.44,2.60,2.44,2.84,3.08,2.92,3.32,3.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vua8j4,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
vunrjc,2.62,2.60,2.48,2.50,2.54,2.40,2.36,2.70,2.40,2.40,...,2.48,2.50,2.36,2.44,2.38,2.50,2.52,2.44,2.66,2.54
vur3vt,4.08,4.20,3.98,3.84,4.06,4.08,3.90,4.10,4.04,4.04,...,4.00,3.82,3.92,3.96,3.96,3.90,3.88,4.02,4.14,4.04
vv73qf,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [ ]:
df_predict.to_csv('저장경로/CBF_total_two_RFR.csv')